# DINOv3 Finetuning Tutorial (LoRA)

This tutorial demonstrates how to efficiently finetune DINOv3 using **Low-Rank Adaptation (LoRA)**.

Ideally suited for adapting the powerful DINOv3 features to downstream tasks like classification or segmentation with minimal parameter updates.

**Agenda:**
1. **Setup**: Load model and apply LoRA.
2. **Training Loop**: Simulate a training process with an optimizer and loss tracking.
3. **Analysis**: Plot the loss curve.
4. **Inference**: Run inference with the adapted model.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from dinov3production import create_model
from dinov3production.finetune.peft import wrap_with_lora

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 1. Model Setup

In [ ]:
# Load Base Model
model = create_model('dinov3_vitb14', pretrained=True).to(device)

# Apply LoRA with rank=16
peft_model = wrap_with_lora(model, r=16)
peft_model.to(device)

# Verify Trainable Parameters
peft_model.print_trainable_parameters()

## 2. Simulated Training Loop
We will fine-tune the model on a dummy classification task (10 classes).

In [ ]:
# Add a classifier head for demonstration (Linear Probe style)
# Note: In a real scenario, you'd attach a head. Here we assume the model output is features.
# Let's wrap it in a simple Module to include a head.
class Classifier(nn.Module):
    def __init__(self, backbone, num_classes=10):
        super().__init__()
        self.backbone = backbone
        self.head = nn.Linear(backbone.embed_dim, num_classes)
        
    def forward(self, x):
        features = self.backbone(x)
        # Assuming backbone returns [B, D] or [B, N, D]. If [B, N, D], take CLS.
        if features.dim() == 3:
            features = features[:, 0]
        return self.head(features)

finetune_model = Classifier(peft_model).to(device)

# Train only the LoRA params and the head
optimizer = optim.AdamW([
    {'params': peft_model.parameters(), 'lr': 1e-4},
    {'params': finetune_model.head.parameters(), 'lr': 1e-3}
])
criterion = nn.CrossEntropyLoss()

# Dummy Data Loop
losses = []
print("Starting Training...")
for step in range(50):
    # Dummy Batch
    inputs = torch.randn(4, 3, 224, 224).to(device)
    targets = torch.randint(0, 10, (4,)).to(device)
    
    optimizer.zero_grad()
    outputs = finetune_model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if step % 10 == 0:
        print(f"Step {step}: Loss {loss.item():.4f}")

print("Training Complete.")

## 3. Analysis
Visualize the training loss.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses, label="Training Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("LoRA Finetuning Loss")
plt.legend()
plt.show()

## 4. Inference
Run the finetuned model on a new input.

In [ ]:
finetune_model.eval()
with torch.no_grad():
    test_input = torch.randn(1, 3, 224, 224).to(device)
    logits = finetune_model(test_input)
    probs = torch.softmax(logits, dim=-1)
    pred_class = torch.argmax(probs, dim=-1).item()
    
print(f"Predicted Class: {pred_class}")
print(f"Probabilities: {probs[0].cpu().numpy().round(2)}")